<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-9.-A-Pythonic-Object" data-toc-modified-id="Chapter-9.-A-Pythonic-Object-1">Chapter 9. A Pythonic Object</a></span><ul class="toc-item"><li><span><a href="#@classmethod-vs-@staticmethod" data-toc-modified-id="@classmethod-vs-@staticmethod-1.1"><code>@classmethod</code> vs <code>@staticmethod</code></a></span></li><li><span><a href="#Hashable-objects" data-toc-modified-id="Hashable-objects-1.2">Hashable objects</a></span></li><li><span><a href="#Private-and-protected-attributes" data-toc-modified-id="Private-and-protected-attributes-1.3">Private and protected attributes</a></span><ul class="toc-item"><li><span><a href="#Name-mangling" data-toc-modified-id="Name-mangling-1.3.1">Name mangling</a></span></li><li><span><a href="#Protected-attributes" data-toc-modified-id="Protected-attributes-1.3.2">Protected attributes</a></span></li></ul></li><li><span><a href="#The-__slots__-class-attribute" data-toc-modified-id="The-__slots__-class-attribute-1.4">The <code>__slots__</code> class attribute</a></span><ul class="toc-item"><li><span><a href="#things-to-remember-when-using-__slots__" data-toc-modified-id="things-to-remember-when-using-__slots__-1.4.1">things to remember when using <code>__slots__</code></a></span></li></ul></li><li><span><a href="#Overriding-Class-Attributes" data-toc-modified-id="Overriding-Class-Attributes-1.5">Overriding Class Attributes</a></span></li></ul></li></ul></div>

# Chapter 9. A Pythonic Object

In [325]:
import math
from array import array

class Vector2d:
    typecode = 'd'   # class attribute used when converting Vector2d 
                     # instances to/from bytes.

    def __init__(self, x, y):
        self.__x = float(x) # private read-only member
                           # catches errors early, helpful in case Vector2d 
                           # is called with unsuitable arguments   
        self.__y = float(y)

    def __iter__(self): # this is what makes unpacking work
        return (i for i in (self.__x, self.__y))   

    def __repr__(self): #  builds a string by interpolating the components 
                        #  with {!r} to get their repr
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  
#     By hardcoding the class_name, subclasses would have to overwrite 
# __repr__ just to change the class_name. 
# By reading the name from the type of the instance, __repr__ is safer 
# to inherit.

    def __str__(self):
        return str(tuple(self))   

    def __bytes__(self): # To generate bytes, convert the typecode to bytes 
                         # and concatenate with bytes converted from an array 
                         # built by iterating over the instance

        return (bytes([ord(self.typecode)]) +   
                bytes(array(self.typecode, self)))   

    def __eq__(self, other):
        return tuple(self) == tuple(other)   

    def __abs__(self):
        return math.hypot(self.__x, self.__y)   

    def __bool__(self): # 0.0 magnitude becomes False, nonzero is True
        return bool(abs(self))   
    
#     method that imports a Vector2d from a binary sequence 
#     adapted from array.array.frombytes
    @classmethod   
    def frombytes(cls, octets):   # No self argument; class itself is passed as cls
        typecode = chr(octets[0])   
        memv = memoryview(octets[1:]).cast(typecode)  
        return cls(*memv) # invoke ctor to build a new instance
    
    def __format__(self, fmt_spec=''):
# Use the format built-in to apply the fmt_spec to each vector component,
# building an iterable of formatted strings
        components = (format(c, fmt_spec) for c in self) #
        return '({}, {})'.format(*components)  
    
    @property   # makes x read-only
    def x(self):   
        return self.__x   # Just return self.__x

    @property   # makes y read-only
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.__x, self.__y))  
    
    def __hash__(self):
#       using the bitwise XOR operator (^) 
#       to mix the hashes of the components  
        return hash(self.__x) ^ hash(self.__y)

In [327]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y) # no getter method calls

3.0 4.0


In [328]:
x, y = v1 # A Vector2d can be unpacked to a tuple of variables.

In [329]:
v1

Vector2d(3.0, 4.0)

In [337]:
v1_clone = eval(repr(v1)) # repr emulates the source code 
                          # for constructing the instance.
v1 is v1_clone

False

In [338]:
v1 == v1_clone  # supports comparison with ==

True

In [336]:
print(v1)

(3.0, 4.0)


In [339]:
octets = bytes(v1) # uses the __bytes__ method 
                   # to produce a binary representation.

In [340]:
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [341]:
abs(v1) # uses the __abs__ method

5.0

In [342]:
bool(v1), bool(Vector2d(0, 0)) #  uses the __bool__ method

(True, False)

## `@classmethod` vs `@staticmethod`

`@classmethod` decorator defines a method that operates on the class and not on instances. It changes the way the method is called, so it receives the class itself as the first argument, instead of an instance. The most common use case is for alternative constructors like `frombytes`

`@staticmethod` decorator changes a method so that it receives no special first argument. In essence, a static method is just like a plain function that happens to live in a class body, instead of being defined at the module level. There aren't any compelling use cases for using `@staticmethod`

In [346]:
class Demo:
     @classmethod
     def klassmeth(*args):
         return args  # returns all arguments, including Demo class as 1st arg
     @staticmethod
     def statmeth(*args):
         return args # returns all arguments, no Demo class returned

In [347]:
Demo.klassmeth() 

(__main__.Demo,)

In [348]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [349]:
Demo.statmeth() 

()

In [350]:
Demo.statmeth('spam')

('spam',)

## Hashable objects
To make a user object hashable, implement `__hash__` and `__eq__` and make all attributes private and read-only (using @property).

In [353]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
print(hash(v1), hash(v2))
print(set([v1, v2]))

7 384307168202284039
{Vector2d(3.1, 4.2), Vector2d(3.0, 4.0)}


## Private and protected attributes
private attributes have two leading underscores and zero or at most one trailing underscore. 

### Name mangling
The interpreter stores the name in the instance `__dict__` prefixed with a leading underscore and the class name, so in the Dog class, `__mood` becomes `_Dog__mood`, 

In [354]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

Name mangling is about safety, not security: it’s designed to prevent accidental access and not intentional wrongdoing

In [355]:
v1._Vector2d__x = 4
v1.__dict__

{'_Vector2d__x': 4, '_Vector2d__y': 4.0}

### Protected attributes

Attributes with a single _ prefix are called “protected” in some corners of the Python documentation.

The single underscore prefix has no special meaning to the Python interpreter when used in attribute names, but it’s a very strong convention among Python programmers that you should not access such attributes from outside the class

## The `__slots__` class attribute
By defining `__slots__` in the class, you are telling the interpreter: “These are all the instance attributes in this class.” Python then stores them in a tuple-like structure in each instance, avoiding the memory overhead of the per-instance `__dict__`.

This can make a huge difference in memory usage if your have millions of instances active at the same time.

### things to remember when using `__slots__`
You must remember to redeclare `__slots__` in each subclass, because the inherited attribute is ignored by the interpreter.

Instances will only be able to have the attributes listed in `__slots__`, unless you include `__dict__` in `__slots__` (but doing so may negate the memory savings).

Instances cannot be targets of weak references unless you remember to include `__weakref__` in `__slots__`.


## Overriding Class Attributes
customizing instance with a different values for for attributes from default

In [356]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [357]:
len(dumpd)

17

In [358]:
# change instance typecode value
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [359]:
len(dumpf)

9

In [360]:
# class attribute value is preserved
Vector2d.typecode

'd'

This can also be accomplished by subclassing

In [361]:
class ShortVector2d(Vector2d):  
    typecode = 'f'
sv = ShortVector2d(1.1, 2.2)
bytes(sv)

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [362]:
len(bytes(sv))

9